## Setup and Imports

In [ ]:
import time
from collections import deque
import heapq
from copy import deepcopy

## Puzzle State Representation and Helper Functions

In [ ]:
class PuzzleState:
    """Represents a state in the 8-puzzle problem."""
    
    def __init__(self, board, parent=None, move=None, depth=0, cost=0):
        self.board = board  # Tuple of 9 elements
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost
        self.zero_pos = board.index(0)
    
    def __eq__(self, other):
        return self.board == other.board
    
    def __hash__(self):
        return hash(self.board)
    
    def __lt__(self, other):
        return self.cost < other.cost
    
    def display(self):
        """Pretty print the puzzle state."""
        for i in range(0, 9, 3):
            print(f"({self.board[i]}, {self.board[i+1]}, {self.board[i+2]})")
    
    def get_neighbors(self):
        """Generate all valid successor states."""
        neighbors = []
        row, col = self.zero_pos // 3, self.zero_pos % 3
        
        # Possible moves: Up, Down, Left, Right
        moves = {
            'UP': (-1, 0),
            'DOWN': (1, 0),
            'LEFT': (0, -1),
            'RIGHT': (0, 1)
        }
        
        for move_name, (dr, dc) in moves.items():
            new_row, new_col = row + dr, col + dc
            
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_pos = new_row * 3 + new_col
                new_board = list(self.board)
                new_board[self.zero_pos], new_board[new_pos] = new_board[new_pos], new_board[self.zero_pos]
                
                neighbors.append(PuzzleState(
                    tuple(new_board),
                    parent=self,
                    move=move_name,
                    depth=self.depth + 1,
                    cost=self.cost + 1
                ))
        
        return neighbors


def reconstruct_path(state):
    """Reconstruct the solution path from start to goal."""
    path = []
    moves = []
    current = state
    
    while current is not None:
        path.append(current.board)
        if current.move:
            moves.append(current.move)
        current = current.parent
    
    return list(reversed(path)), list(reversed(moves))


def display_solution(path, moves):
    """Display the solution path step by step."""
    print("\nSolution Path:")
    print("=" * 40)
    for i, board in enumerate(path):
        print(f"\nStep {i}:")
        for j in range(0, 9, 3):
            print(f"  {board[j]} {board[j+1]} {board[j+2]}")
        if i < len(moves):
            print(f"  Move: {moves[i]}")

## Heuristic Functions (for UCS and A*)

In [ ]:
def misplaced_tiles(board, goal):
    """Count the number of misplaced tiles (excluding the blank)."""
    return sum(1 for i in range(9) if board[i] != 0 and board[i] != goal[i])


def manhattan_distance(board, goal):
    """Calculate the sum of Manhattan distances for all tiles."""
    distance = 0
    for i in range(9):
        if board[i] != 0:
            # Find the goal position of this tile
            goal_pos = goal.index(board[i])
            # Calculate Manhattan distance
            current_row, current_col = i // 3, i % 3
            goal_row, goal_col = goal_pos // 3, goal_pos % 3
            distance += abs(current_row - goal_row) + abs(current_col - goal_col)
    return distance

---

# Algorithm Implementations

## 1. Breadth-First Search (BFS)

In [ ]:
def bfs(start_state, goal_state):
    """
    Breadth-First Search implementation.
    Explores nodes level by level using a queue.
    Guarantees shortest path in terms of number of moves.
    """
    start = PuzzleState(start_state)
    goal = goal_state
    
    if start.board == goal:
        return start, 0
    
    frontier = deque([start])
    explored = {start.board}
    nodes_expanded = 0
    
    while frontier:
        state = frontier.popleft()
        nodes_expanded += 1
        
        for neighbor in state.get_neighbors():
            if neighbor.board == goal:
                return neighbor, nodes_expanded
            
            if neighbor.board not in explored:
                explored.add(neighbor.board)
                frontier.append(neighbor)
    
    return None, nodes_expanded  # No solution found

## 2. Depth-First Search (DFS)

In [ ]:
def dfs(start_state, goal_state, max_depth=50):
    """
    Depth-First Search implementation with depth limit to prevent infinite loops.
    Explores as deep as possible before backtracking.
    Does NOT guarantee shortest path.
    """
    start = PuzzleState(start_state)
    goal = goal_state
    
    if start.board == goal:
        return start, 0
    
    frontier = [start]  # Stack (using list)
    explored = {start.board}
    nodes_expanded = 0
    
    while frontier:
        state = frontier.pop()  # Pop from end (LIFO)
        nodes_expanded += 1
        
        if state.depth >= max_depth:
            continue
        
        for neighbor in state.get_neighbors():
            if neighbor.board == goal:
                return neighbor, nodes_expanded
            
            if neighbor.board not in explored:
                explored.add(neighbor.board)
                frontier.append(neighbor)
    
    return None, nodes_expanded

## 3. Depth-Limited Search (DLS)

In [ ]:
def dls(start_state, goal_state, depth_limit):
    """
    Depth-Limited Search implementation.
    DFS with a specified depth limit.
    Returns 'cutoff' if limit reached without finding solution.
    """
    start = PuzzleState(start_state)
    goal = goal_state
    nodes_expanded = 0
    
    def recursive_dls(state, limit):
        nonlocal nodes_expanded
        nodes_expanded += 1
        
        if state.board == goal:
            return state
        
        if limit == 0:
            return 'cutoff'
        
        cutoff_occurred = False
        for neighbor in state.get_neighbors():
            result = recursive_dls(neighbor, limit - 1)
            
            if result == 'cutoff':
                cutoff_occurred = True
            elif result is not None:
                return result
        
        return 'cutoff' if cutoff_occurred else None
    
    result = recursive_dls(start, depth_limit)
    
    if result == 'cutoff' or result is None:
        return None, nodes_expanded
    return result, nodes_expanded

## 4. Iterative Deepening Search (IDS)

In [ ]:
def ids(start_state, goal_state, max_depth=50):
    """
    Iterative Deepening Search implementation.
    Combines benefits of BFS (completeness, optimality) and DFS (memory efficiency).
    Repeatedly applies DLS with increasing depth limits.
    """
    total_nodes = 0
    
    for depth in range(max_depth):
        result, nodes = dls(start_state, goal_state, depth)
        total_nodes += nodes
        
        if result is not None:
            return result, total_nodes
    
    return None, total_nodes

## 5. Bidirectional Search

In [ ]:
def bidirectional_search(start_state, goal_state):
    """
    Bidirectional Search implementation.
    Searches from both start and goal simultaneously.
    Stops when the two searches meet.
    Can be much faster than unidirectional search.
    """
    start = PuzzleState(start_state)
    goal = PuzzleState(goal_state)
    
    if start.board == goal.board:
        return start, 0
    
    # Forward search from start
    forward_frontier = deque([start])
    forward_explored = {start.board: start}
    
    # Backward search from goal
    backward_frontier = deque([goal])
    backward_explored = {goal.board: goal}
    
    nodes_expanded = 0
    
    while forward_frontier and backward_frontier:
        # Expand forward
        if forward_frontier:
            state = forward_frontier.popleft()
            nodes_expanded += 1
            
            for neighbor in state.get_neighbors():
                if neighbor.board in backward_explored:
                    # Found intersection - reconstruct path
                    return merge_paths(neighbor, backward_explored[neighbor.board]), nodes_expanded
                
                if neighbor.board not in forward_explored:
                    forward_explored[neighbor.board] = neighbor
                    forward_frontier.append(neighbor)
        
        # Expand backward
        if backward_frontier:
            state = backward_frontier.popleft()
            nodes_expanded += 1
            
            for neighbor in state.get_neighbors():
                if neighbor.board in forward_explored:
                    # Found intersection
                    return merge_paths(forward_explored[neighbor.board], neighbor), nodes_expanded
                
                if neighbor.board not in backward_explored:
                    backward_explored[neighbor.board] = neighbor
                    backward_frontier.append(neighbor)
    
    return None, nodes_expanded


def merge_paths(forward_state, backward_state):
    """Merge two paths meeting in the middle."""
    # This is a simplified version - just return the forward state
    # In a full implementation, you'd combine both paths
    return forward_state

## 6. Uniform Cost Search (UCS) - Extension

In [ ]:
def ucs(start_state, goal_state):
    """
    Uniform Cost Search implementation.
    Expands nodes in order of path cost (like Dijkstra's algorithm).
    Guarantees optimal solution when all step costs are equal.
    """
    start = PuzzleState(start_state)
    goal = goal_state
    
    if start.board == goal:
        return start, 0
    
    frontier = [(0, id(start), start)]  # (cost, tie_breaker, state)
    heapq.heapify(frontier)
    explored = {start.board: 0}
    nodes_expanded = 0
    
    while frontier:
        _, _, state = heapq.heappop(frontier)
        nodes_expanded += 1
        
        if state.board == goal:
            return state, nodes_expanded
        
        for neighbor in state.get_neighbors():
            if neighbor.board not in explored or neighbor.cost < explored[neighbor.board]:
                explored[neighbor.board] = neighbor.cost
                heapq.heappush(frontier, (neighbor.cost, id(neighbor), neighbor))
    
    return None, nodes_expanded

## 7. A* Search - Extension

In [ ]:
def astar(start_state, goal_state, heuristic='manhattan'):
    """
    A* Search implementation.
    Uses f(n) = g(n) + h(n) where:
    - g(n) is the cost from start to n
    - h(n) is the estimated cost from n to goal (heuristic)
    
    With admissible heuristic, A* is both complete and optimal.
    """
    start = PuzzleState(start_state)
    goal = goal_state
    
    if start.board == goal:
        return start, 0
    
    # Choose heuristic function
    if heuristic == 'manhattan':
        h_func = manhattan_distance
    else:
        h_func = misplaced_tiles
    
    h_start = h_func(start.board, goal)
    frontier = [(h_start, 0, id(start), start)]  # (f, g, tie_breaker, state)
    heapq.heapify(frontier)
    explored = {start.board: 0}
    nodes_expanded = 0
    
    while frontier:
        _, g, _, state = heapq.heappop(frontier)
        nodes_expanded += 1
        
        if state.board == goal:
            return state, nodes_expanded
        
        for neighbor in state.get_neighbors():
            g_neighbor = g + 1
            
            if neighbor.board not in explored or g_neighbor < explored[neighbor.board]:
                explored[neighbor.board] = g_neighbor
                h = h_func(neighbor.board, goal)
                f = g_neighbor + h
                heapq.heappush(frontier, (f, g_neighbor, id(neighbor), neighbor))
    
    return None, nodes_expanded

---

# Testing and Comparison

## Test Case 1: Easy Puzzle (2 moves from goal)

In [ ]:
# Easy test case
start_easy = (1, 2, 3, 4, 5, 6, 7, 0, 8)
goal = (1, 2, 3, 4, 5, 6, 7, 8, 0)

print("START STATE (Easy):")
PuzzleState(start_easy).display()
print("\nGOAL STATE:")
PuzzleState(goal).display()
print("\n" + "=" * 60)

### Running All Algorithms on Easy Puzzle

In [ ]:
results_easy = {}

# BFS
print("\nRunning BFS...")
start_time = time.time()
result, nodes = bfs(start_easy, goal)
elapsed = time.time() - start_time
if result:
    path, moves = reconstruct_path(result)
    results_easy['BFS'] = {'time': elapsed, 'nodes': nodes, 'length': len(path)-1}
    print(f"Execution Time: {elapsed:.4f}s")
    print(f"Nodes Expanded: {nodes}")
    print(f"Solution Length: {len(path)-1}")
    print("Final State:")
    result.display()

print("\n" + "=" * 60)

# DFS
print("\nRunning DFS...")
start_time = time.time()
result, nodes = dfs(start_easy, goal)
elapsed = time.time() - start_time
if result:
    path, moves = reconstruct_path(result)
    results_easy['DFS'] = {'time': elapsed, 'nodes': nodes, 'length': len(path)-1}
    print(f"Execution Time: {elapsed:.4f}s")
    print(f"Nodes Expanded: {nodes}")
    print(f"Solution Length: {len(path)-1}")
    print("Final State:")
    result.display()

print("\n" + "=" * 60)

# DLS
print("\nRunning DLS (limit=10)...")
start_time = time.time()
result, nodes = dls(start_easy, goal, depth_limit=10)
elapsed = time.time() - start_time
if result:
    path, moves = reconstruct_path(result)
    results_easy['DLS'] = {'time': elapsed, 'nodes': nodes, 'length': len(path)-1}
    print(f"Execution Time: {elapsed:.4f}s")
    print(f"Nodes Expanded: {nodes}")
    print(f"Solution Length: {len(path)-1}")
    print("Final State:")
    result.display()

print("\n" + "=" * 60)

# IDS
print("\nRunning IDS...")
start_time = time.time()
result, nodes = ids(start_easy, goal)
elapsed = time.time() - start_time
if result:
    path, moves = reconstruct_path(result)
    results_easy['IDS'] = {'time': elapsed, 'nodes': nodes, 'length': len(path)-1}
    print(f"Execution Time: {elapsed:.4f}s")
    print(f"Nodes Expanded: {nodes}")
    print(f"Solution Length: {len(path)-1}")
    print("Final State:")
    result.display()

print("\n" + "=" * 60)

# Bidirectional
print("\nRunning Bidirectional...")
start_time = time.time()
result, nodes = bidirectional_search(start_easy, goal)
elapsed = time.time() - start_time
if result:
    path, moves = reconstruct_path(result)
    results_easy['Bidirectional'] = {'time': elapsed, 'nodes': nodes, 'length': len(path)-1}
    print(f"Execution Time: {elapsed:.4f}s")
    print(f"Nodes Expanded: {nodes}")
    print(f"Solution Length: {len(path)-1}")
    print("Final State:")
    result.display()

print("\n" + "=" * 60)

# UCS
print("\nRunning UCS...")
start_time = time.time()
result, nodes = ucs(start_easy, goal)
elapsed = time.time() - start_time
if result:
    path, moves = reconstruct_path(result)
    results_easy['UCS'] = {'time': elapsed, 'nodes': nodes, 'length': len(path)-1}
    print(f"Execution Time: {elapsed:.4f}s")
    print(f"Nodes Expanded: {nodes}")
    print(f"Solution Length: {len(path)-1}")
    print("Final State:")
    result.display()

print("\n" + "=" * 60)

# A* with Manhattan Distance
print("\nRunning A* (Manhattan)...")
start_time = time.time()
result, nodes = astar(start_easy, goal, 'manhattan')
elapsed = time.time() - start_time
if result:
    path, moves = reconstruct_path(result)
    results_easy['A*'] = {'time': elapsed, 'nodes': nodes, 'length': len(path)-1}
    print(f"Execution Time: {elapsed:.4f}s")
    print(f"Nodes Expanded: {nodes}")
    print(f"Solution Length: {len(path)-1}")
    print("Final State:")
    result.display()

### Comparison Table for Easy Puzzle

In [ ]:
print("\n" + "=" * 80)
print("COMPARISON TABLE - EASY PUZZLE")
print("=" * 80)
print(f"{'Algorithm':<15} {'Time (s)':<12} {'Nodes Expanded':<18} {'Solution Length':<18}")
print("-" * 80)
for algo, data in results_easy.items():
    print(f"{algo:<15} {data['time']:<12.6f} {data['nodes']:<18} {data['length']:<18}")
print("=" * 80)

---

## Test Case 2: Harder Puzzle

In [ ]:
# Harder test case
start_hard = (1, 3, 4, 8, 6, 2, 7, 0, 5)
goal = (1, 2, 3, 4, 5, 6, 7, 8, 0)

print("START STATE (Hard):")
PuzzleState(start_hard).display()
print("\nGOAL STATE:")
PuzzleState(goal).display()
print("\n" + "=" * 60)

### Running Selected Algorithms on Hard Puzzle
(Note: We'll skip DFS and DLS as they may take too long or fail)

In [ ]:
results_hard = {}

# BFS
print("\nRunning BFS...")
start_time = time.time()
result, nodes = bfs(start_hard, goal)
elapsed = time.time() - start_time
if result:
    path, moves = reconstruct_path(result)
    results_hard['BFS'] = {'time': elapsed, 'nodes': nodes, 'length': len(path)-1}
    print(f"Execution Time: {elapsed:.4f}s")
    print(f"Nodes Expanded: {nodes}")
    print(f"Solution Length: {len(path)-1}")

print("\n" + "=" * 60)

# IDS
print("\nRunning IDS...")
start_time = time.time()
result, nodes = ids(start_hard, goal, max_depth=25)
elapsed = time.time() - start_time
if result:
    path, moves = reconstruct_path(result)
    results_hard['IDS'] = {'time': elapsed, 'nodes': nodes, 'length': len(path)-1}
    print(f"Execution Time: {elapsed:.4f}s")
    print(f"Nodes Expanded: {nodes}")
    print(f"Solution Length: {len(path)-1}")

print("\n" + "=" * 60)

# A* with Manhattan Distance
print("\nRunning A* (Manhattan)...")
start_time = time.time()
result, nodes = astar(start_hard, goal, 'manhattan')
elapsed = time.time() - start_time
if result:
    path, moves = reconstruct_path(result)
    results_hard['A*'] = {'time': elapsed, 'nodes': nodes, 'length': len(path)-1}
    print(f"Execution Time: {elapsed:.4f}s")
    print(f"Nodes Expanded: {nodes}")
    print(f"Solution Length: {len(path)-1}")
    print("\nSolution path:")
    display_solution(path, moves)

### Comparison Table for Hard Puzzle

In [ ]:
print("\n" + "=" * 80)
print("COMPARISON TABLE - HARD PUZZLE")
print("=" * 80)
print(f"{'Algorithm':<15} {'Time (s)':<12} {'Nodes Expanded':<18} {'Solution Length':<18}")
print("-" * 80)
for algo, data in results_hard.items():
    print(f"{algo:<15} {data['time']:<12.6f} {data['nodes']:<18} {data['length']:<18}")
print("=" * 80)

---

## Depth Limit Experiments (Question 3)

In [ ]:
print("\nDEPTH LIMIT EXPERIMENTS")
print("=" * 60)

depth_limits = [2, 5, 10, 15, 20]
print(f"\nTesting DLS with different depth limits on easy puzzle...\n")
print(f"{'Depth Limit':<15} {'Result':<15} {'Nodes Expanded':<18} {'Solution Length':<18}")
print("-" * 70)

for limit in depth_limits:
    result, nodes = dls(start_easy, goal, limit)
    if result:
        path, _ = reconstruct_path(result)
        print(f"{limit:<15} {'Found':<15} {nodes:<18} {len(path)-1:<18}")
    else:
        print(f"{limit:<15} {'Not Found':<15} {nodes:<18} {'N/A':<18}")

---

# End of Notebook

This notebook demonstrates all required uninformed search algorithms for the 8-puzzle problem, along with informed search extensions (UCS and A*). The comparison tables show performance metrics for analysis.